In [1]:
# YOU CAN SIMPLY RUN THIS FILE TO DOWNLOAD THE LATEST DATA FOR A TICKER

# We are extracting the date and OHLCV (open, high, low, close, volume) of a stock for a given time period.
# This data is meant to be used in combination with TA, but can be used standalone if desired.

In [2]:
import yfinance as yf
from ta import momentum
from ta import volume
from ta import volatility
from ta import trend
from ta import add_all_ta_features
import pandas as pd
import numpy as np

np.seterr(divide='ignore', invalid='ignore')    # remove warning that appears when calculating some TA. Known issue from library

{'divide': 'warn', 'over': 'warn', 'under': 'ignore', 'invalid': 'warn'}

In [3]:
# To add a new TA column, simply follow the examples below. Some TA might have more than one function, i.e. MACD and MACD signal.
# Use the ta library documentation to find more indicators for each of the below categories.
# Hover over the TA class to see what data is needed: open, high, low, close, volume
# Some TA classes have extra arguments like window size.

In [4]:
# Momentum
def add_momentum_indicators(df: pd.DataFrame):
    df['rsi'] =         momentum.RSIIndicator(df['Close'], window=14).rsi()
    df['stoch_rsi'] =   momentum.StochRSIIndicator(df['Close'], window=14).stochrsi()
    df['stoch_rsi_d'] = momentum.StochRSIIndicator(df['Close'], window=14).stochrsi_d()
    df['stoch_rsi_k'] = momentum.StochRSIIndicator(df['Close'], window=14).stochrsi_k()
    df['stoch_osc'] =   momentum.StochasticOscillator(df['High'], df['Low'], df['Close'], window=14, smooth_window=3).stoch()
    df['awesome_osc'] = momentum.AwesomeOscillatorIndicator(df['High'], df['Low'], window1=5, window2=34).awesome_oscillator()

    return df

In [5]:
# Volume
def add_volume_indicators(df: pd.DataFrame):
    df['force_index'] =         volume.ForceIndexIndicator(df['Close'], df['Volume'], window=13).force_index()
    df['ease_move'] =           volume.EaseOfMovementIndicator(df['High'], df['Low'], df['Volume'], window=14).ease_of_movement()
    df['ease_move_signal'] =    volume.EaseOfMovementIndicator(df['High'], df['Low'], df['Volume'], window=14).sma_ease_of_movement()
    df['obv'] =                 volume.OnBalanceVolumeIndicator(df['Close'], df['Volume']).on_balance_volume()
    df['adi'] =                 volume.AccDistIndexIndicator(df['High'], df['Low'], df['Close'], df['Volume']).acc_dist_index()

    return df

In [6]:
# Volatility
def add_volatility_indicators(df: pd.DataFrame):
    df['bbands_upper'] =    volatility.BollingerBands(df['Close'], window=20).bollinger_hband()
    df['bbands_middle'] =   volatility.BollingerBands(df['Close'], window=20).bollinger_mavg()
    df['bbands_lower'] =    volatility.BollingerBands(df['Close'], window=20).bollinger_lband()
    df['atr'] =             volatility.AverageTrueRange(df['High'], df['Low'], df['Close'], window=14).average_true_range()

    return df

In [7]:
# Trend
def add_trend_indicators(df: pd.DataFrame):
    df['macd'] =        trend.MACD(df['Close'], window_slow=26, window_fast=12, window_sign=9).macd()
    df['macd_signal'] = trend.MACD(df['Close'], window_slow=26, window_fast=12, window_sign=9).macd_signal()
    df['aroon'] =       trend.AroonIndicator(df['Close'], window=25).aroon_indicator()
    df['aroon_up'] =    trend.AroonIndicator(df['Close'], window=25).aroon_up()
    df['aroon_down'] =  trend.AroonIndicator(df['Close'], window=25).aroon_down()
    df['adx'] =         trend.ADXIndicator(df['High'], df['Low'], df['Close'], window=14).adx()
    df['adx_neg'] =     trend.ADXIndicator(df['High'], df['Low'], df['Close'], window=14).adx_neg()
    df['adx_pos'] =     trend.ADXIndicator(df['High'], df['Low'], df['Close'], window=14).adx_pos()
    df['ema_50'] =      trend.EMAIndicator(df['Close'], window=50).ema_indicator()
    df['ema_200'] =     trend.EMAIndicator(df['Close'], window=200).ema_indicator()
    df['sma_50'] =      trend.SMAIndicator(df['Close'], window=50).sma_indicator()
    df['sma_200'] =     trend.SMAIndicator(df['Close'], window=200).sma_indicator()
   
    return df

In [8]:
ticker = 'QQQ'
start_date = '2013-10-13'
end_date = '2023-11-25' # you can set end date to a future date to download all possible data.
file_location = 'data/ta_stock_data_' + ticker + '.csv'

In [9]:
t_hist = yf.download(ticker, start=start_date, end=end_date)

[*********************100%%**********************]  1 of 1 completed


In [10]:
# Isolating the important columns from dataframe
df: pd.DataFrame = t_hist[['Open', 'High', 'Low', 'Close', 'Volume']]

# Add (or don't add) batches of indicators here.
df = add_momentum_indicators(df)
df = add_volume_indicators(df)
df = add_volatility_indicators(df)
df = add_trend_indicators(df)

# df = add_all_ta_features(df, df['Open'], df['High'], df['Low'], df['Close'], df['Volume'])    # Nuclear option if you are lazy.

df = df.round(8)
df = df.dropna()    # important to note that the first x rows will be dropped, where x is the largest window size from the TA indicators above.

print(df.shape)
print(df.isnull().sum())

df

(2331, 32)
Open                0
High                0
Low                 0
Close               0
Volume              0
rsi                 0
stoch_rsi           0
stoch_rsi_d         0
stoch_rsi_k         0
stoch_osc           0
awesome_osc         0
force_index         0
ease_move           0
ease_move_signal    0
obv                 0
adi                 0
bbands_upper        0
bbands_middle       0
bbands_lower        0
atr                 0
macd                0
macd_signal         0
aroon               0
aroon_up            0
aroon_down          0
adx                 0
adx_neg             0
adx_pos             0
ema_50              0
ema_200             0
sma_50              0
sma_200             0
dtype: int64


,Open,High,Low,Close,Volume,rsi,stoch_rsi,stoch_rsi_d,stoch_rsi_k,stoch_osc,...,aroon,aroon_up,aroon_down,adx,adx_neg,adx_pos,ema_50,ema_200,sma_50,sma_200
Date,,,,,,,,,,,,,,,,,,,,,
2014-07-30,97.139999,97.320000,96.599998,96.980003,35348500,64.825687,0.643653,0.573199,0.576712,83.385631,...,76.0,80.0,4.0,18.788158,22.101037,23.358166,93.752395,88.772826,93.556800,88.25180
2014-07-31,96.180000,96.339996,94.879997,95.019997,50047900,48.422723,0.000000,0.503912,0.375157,21.943461,...,72.0,76.0,4.0,19.102935,31.548004,19.668354,93.802104,88.834987,93.697200,88.32795
2014-08-01,94.820000,95.339996,94.040001,94.669998,60708000,46.175826,0.000000,0.388807,0.214551,18.155534,...,68.0,72.0,4.0,20.038630,34.698553,17.794416,93.836139,88.893047,93.813800,88.40390
2014-08-04,94.870003,95.690002,94.519997,95.320000,33230500,50.746729,0.202050,0.219019,0.067350,36.887560,...,64.0,68.0,4.0,20.435381,31.765168,18.819072,93.894330,88.956997,93.935600,88.47860
2014-08-05,94.940002,95.209999,94.150002,94.589996,50331100,46.019944,0.000000,0.116417,0.067350,15.850007,...,44.0,64.0,20.0,21.089999,31.753351,17.248721,93.921611,89.013046,94.029800,88.54735
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-10-25,356.700012,356.880005,349.750000,350.339996,71190500,36.780938,0.000000,0.136813,0.131498,2.459344,...,-40.0,60.0,100.0,20.398628,30.902373,11.841108,363.936113,345.359392,365.438399,338.59195
2023-10-26,349.119995,350.070007,342.350006,343.660004,78621400,32.123765,0.000000,0.127222,0.105863,4.173298,...,-44.0,56.0,100.0,22.830027,36.497913,10.767478,363.140971,345.342482,365.060799,338.94610
2023-10-27,346.429993,348.839996,344.079987,345.309998,63420200,34.335442,0.085174,0.088584,0.028391,9.429732,...,-44.0,52.0,96.0,25.087754,34.325009,10.126436,362.441717,345.342159,364.795399,339.28490


In [11]:
df.to_csv(file_location)